<a href="https://colab.research.google.com/github/respect5716/webshooter/blob/main/examples/naver-news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naver News

## 1. Setting

In [ ]:
!pip install webshooter
!pip install newspaper3k

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article

from webshooter import StaticScraper, url_to_soup

In [ ]:
import logging
logging.root.setLevel(logging.INFO)

## 2. Scrape News List

### Define scraper

In [ ]:
config = {
    'date': '20211015',
    'max_page': 1,
    'categories': ['정치', '경제', '사회', '세계', '생활/문화', 'IT/과학'],
    'category_to_id': {
        '정치': '100',
        '경제': '101',
        '사회': '102',
        '생활/문화': '103',
        '세계': '104',
        'IT/과학': '105',
    }
}

In [ ]:
list_app = StaticScraper(progbar = True)
list_app.set_vars(config)

### Define funcs

In [ ]:
def get_page_url(category, date, page):
    category_id = list_app.v.category_to_id[category]
    url = f'https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1={category_id}&listType=title&date={date}&page={page}'
    return url

def find_max_page(category):
    page_url = get_page_url(category, list_app.v.date, 10000)
    soup = url_to_soup(page_url)
    max_page = soup.select('div.paging strong')[0].text
    return min(list_app.v.max_page, int(max_page))

@list_app.register('browse')
def browse():
    urls = []
    for category in list_app.v.categories:
        max_page = find_max_page(category)
        urls += [get_page_url(category, list_app.v.date, p) for p in range(1, max_page+1)]
    return urls

@list_app.register('parse', multiprocess=True)
def parse(html):
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.select('div.list_body ul.type02 a')
    titles = [i.text for i in articles]
    urls = [i.get('href') for i in articles]
    authors = [i.text for i in soup.select('div.list_body ul.type02 span.writing')]
    return [{'title': t, 'author': a, 'url':u} for t, a, u in zip(titles, authors, urls)]

### Run

In [ ]:
meta = list_app.run()
meta.head()

INFO:root:All functions are registered!
INFO:root:Browsing started
INFO:root:Browsing finished
INFO:root:Requesting started


request:   0%|          | 0/6 [00:00<?, ?it/s]

INFO:root:Requesting finished
INFO:root:Parsing started


parse:   0%|          | 0/6 [00:00<?, ?it/s]

INFO:root:Parsing finished
INFO:root:Merging started
INFO:root:Merging finished
INFO:root:Postprocessing started
INFO:root:Postprocessing finished


,title,author,url
0,“원팀” 외친 이재명 설훈 포옹…尹에는 “후보 사퇴하라”,KBS,https://news.naver.com/main/read.naver?mode=LS...
1,윤석열 “손바닥에 民 써야하나”…이준석 “나라면 洪 쓰겠다”,서울신문,https://news.naver.com/main/read.naver?mode=LS...
2,"""이재명 재선 땐 대장동 사업 빨라질 것""…남욱 녹음파일 공개",뉴스1,https://news.naver.com/main/read.naver?mode=LS...
3,"미 해군, 코로나 백신 접종 거부자에 강제전역 '극약처방'",연합뉴스,https://news.naver.com/main/read.naver?mode=LS...
4,"홍준표 ""정치 4개월 하고 대통령?"" vs 윤석열 ""26년 정치하고 왜 그러나?""",프레시안,https://news.naver.com/main/read.naver?mode=LS...


## 3. Scrape Article

### Define scraper

In [ ]:
article_app = StaticScraper(progbar = True)
article_app.set_var('meta', meta)

### Define funcs

In [ ]:
@article_app.register('browse')
def browse_article():
    return article_app.v.meta['url'].to_list()
    
@article_app.register('parse', multiprocess=True)
def parse_article(html):
    article = Article('', language='ko', fetch_images=False)
    article.download(html)
    article.parse()
    text = article.text
    return {'text': text}
    

@article_app.register('postprocess')
def postprocess(table):
    data = pd.concat([article_app.v.meta, table], axis=1)
    return data

### Run

In [ ]:
data = article_app.run()
data.head()

INFO:root:All functions are registered!
INFO:root:Browsing started
INFO:root:Browsing finished
INFO:root:Requesting started


request:   0%|          | 0/300 [00:00<?, ?it/s]

INFO:root:Requesting finished
INFO:root:Parsing started


parse:   0%|          | 0/300 [00:00<?, ?it/s]

INFO:root:Parsing finished
INFO:root:Merging started
INFO:root:Merging finished
INFO:root:Postprocessing started
INFO:root:Postprocessing finished


,title,author,url,text
0,“원팀” 외친 이재명 설훈 포옹…尹에는 “후보 사퇴하라”,KBS,https://news.naver.com/main/read.naver?mode=LS...,[앵커]민주당 이재명 후보는 후보 선출 뒤 처음으로 당 소속 의원들과 상견례를 갖고...
1,윤석열 “손바닥에 民 써야하나”…이준석 “나라면 洪 쓰겠다”,서울신문,https://news.naver.com/main/read.naver?mode=LS...,지난 1일 서울 중구 필동 매경미디어센터에서 열린 대선 경선 5차 방송토론회에 참석...
2,"""이재명 재선 땐 대장동 사업 빨라질 것""…남욱 녹음파일 공개",뉴스1,https://news.naver.com/main/read.naver?mode=LS...,© News1 이재명 기자 © News1 이재명 기자\n\n(서울=뉴스1) 김민성 ...
3,"미 해군, 코로나 백신 접종 거부자에 강제전역 '극약처방'",연합뉴스,https://news.naver.com/main/read.naver?mode=LS...,지난해 코로나 집단 발병한 미 해군 항공모함 시어도어 루스벨트호\n\n[미 해군/A...
4,"홍준표 ""정치 4개월 하고 대통령?"" vs 윤석열 ""26년 정치하고 왜 그러나?""",프레시안,https://news.naver.com/main/read.naver?mode=LS...,▲국민의힘 대선후보 경선 주자들인 윤석열 전 검찰총장과 홍준표 의원이 15일 밤 1...
